Extraer BTC en términos de 4H

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
def fetch_binance_data_4h(symbol, start_date):
    """
    Descarga datos históricos de Binance en intervalos de 4 horas.

    Args:
        symbol (str): El par de trading, por ejemplo, "BTCUSDT".
        start_date (str): Fecha de inicio en formato "YYYY-MM-DD".
    
    Returns:
        pd.DataFrame: DataFrame con los datos históricos en intervalos de 4 horas.
    """
    base_url = "https://api.binance.com/api/v3/klines"
    interval = "4h"
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    end_time = int(datetime.now().timestamp() * 1000)

    data = []
    while start_time < end_time:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": start_time,
            "limit": 1000  # Máximo permitido por la API
        }
        response = requests.get(base_url, params=params)
        response_data = response.json()
        
        # Verificar si la respuesta contiene datos
        if not response_data:
            break

        data.extend(response_data)
        start_time = response_data[-1][6] + 1  # Actualiza el tiempo de inicio al siguiente intervalo
    
    # Crear DataFrame
    columns = [
        "Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time",
        "Quote Asset Volume", "Number of Trades", "Taker Buy Base Volume",
        "Taker Buy Quote Volume", "Ignore"
    ]
    df = pd.DataFrame(data, columns=columns)

    # Convertir timestamps a fechas legibles
    df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
    df["Close Time"] = pd.to_datetime(df["Close Time"], unit="ms")

    # Convertir columnas numéricas
    numeric_cols = ["Open", "High", "Low", "Close", "Volume", "Quote Asset Volume", "Taker Buy Base Volume", "Taker Buy Quote Volume"]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

    return df

In [4]:
# Descargar datos de BTCUSDT en intervalos de 4 horas desde el inicio
symbol = "BTCUSDT"
start_date = "2017-08-17"  # Fecha inicial de trading para BTC en Binance
btc_4h_data = fetch_binance_data_4h(symbol, start_date)

In [5]:
# Guardar en un archivo CSV
btc_4h_data.to_csv("BTC_4H_data.csv", index=False)
print("Datos guardados en 'BTC_4H_data.csv'.")

Datos guardados en 'BTC_4H_data.csv'.
